In [1]:
import pandas as pd

df = pd.read_csv('../Data Processed/Cleaned Reviews.csv')
df.head()


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,HelpfulnessRatio,ReviewLength
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,0.50,263
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0.00,190
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,0.50,509
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,0.75,219
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,0.00,140


Create high-level business metrics

In [2]:
summary = {
    "total_reviews": len(df),
    "avg_rating": df['Score'].mean(),
    "percent_positive": round(len(df[df['Score'] >= 4]) / len(df) * 100, 2),
    "percent_negative": round(len(df[df['Score'] <= 2]) / len(df) * 100, 2),
    "avg_review_length": df['ReviewLength'].mean(),
    "avg_helpfulness_ratio": df['HelpfulnessRatio'].mean()
}

summary_df = pd.DataFrame([summary])
summary_df


,total_reviews,avg_rating,percent_positive,percent_negative,avg_review_length,avg_helpfulness_ratio
0,568454,4.183199,78.07,14.43,436.222083,0.273433


Create breakdowns by rating category

In [3]:
rating_summary = df.groupby('Score').agg({
    'ReviewLength': 'mean',
    'HelpfulnessRatio': 'mean',
    'Text': 'count'
}).rename(columns={'Text': 'ReviewCount'})

rating_summary.reset_index(inplace=True)
rating_summary


,Score,ReviewLength,HelpfulnessRatio,ReviewCount
0,1,478.082364,0.307190,52268
1,2,490.655010,0.256389,29769
2,3,520.319841,0.242242,42640
3,4,496.603273,0.247915,80655
4,5,402.447373,0.279302,363122


Combine into one exportable dataset

In [4]:
export_df = rating_summary.copy()
export_df['PercentOfTotal'] = round(export_df['ReviewCount'] / len(df) * 100, 2)

export_df


,Score,ReviewLength,HelpfulnessRatio,ReviewCount,PercentOfTotal
0,1,478.082364,0.307190,52268,9.19
1,2,490.655010,0.256389,29769,5.24
2,3,520.319841,0.242242,42640,7.50
3,4,496.603273,0.247915,80655,14.19
4,5,402.447373,0.279302,363122,63.88


Save the summary metrics file


In [5]:
export_df.to_csv('../Dashboards/Summary Metrics.csv', index=False)

In [6]:
df['Sentiment'] = df['Score'].apply(lambda x: 'Positive' if x >= 4 else ('Negative' if x <= 2 else 'Neutral'))
sentiment_summary = df.groupby('Sentiment').size().reset_index(name='Count')
sentiment_summary['Percent'] = round(sentiment_summary['Count'] / len(df) * 100, 2)

sentiment_summary.to_csv('../Dashboards/Sentiment Summary.csv', index=False)


In [7]:
import pandas as pd

df = pd.read_csv('../Data Processed/Cleaned Reviews.csv')
df = df[df['Score'] != 3]
df['Sentiment'] = df['Score'].apply(lambda x: 'Positive' if x >= 4 else 'Negative')


Sentiment distribution over time

In [8]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

sentiment_over_time = df.set_index('Time').resample('M')['Sentiment'].value_counts().unstack().fillna(0)
sentiment_over_time.to_csv('../Dashboards/Sentiment_over_time.csv')


/var/folders/z2/rcq805356td52ljv0dcymspw0000gn/T/ipykernel_15190/835250016.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sentiment_over_time = df.set_index('Time').resample('M')['Sentiment'].value_counts().unstack().fillna(0)


Average rating over time

In [9]:
rating_over_time = df.set_index('Time').resample('M')['Score'].mean().reset_index()
rating_over_time.to_csv('../Dashboards/Rating_over_time.csv', index=False)


/var/folders/z2/rcq805356td52ljv0dcymspw0000gn/T/ipykernel_15190/1528822042.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rating_over_time = df.set_index('Time').resample('M')['Score'].mean().reset_index()


Helpful vs unhelpful

In [10]:
df['IsHelpful'] = df['HelpfulnessRatio'] > 0.5
helpfulness_summary = df.groupby(['Sentiment', 'IsHelpful']).size().reset_index(name='Count')
helpfulness_summary.to_csv('../Dashboards/Helpfulness_summary.csv', index=False)
